<a href="https://colab.research.google.com/github/cwbeitel/clarify/blob/add-colab-nb/clarify/research/image_fec/docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning expression representations

Contributors: @cwbeitel

# Summary

TODO: Image FEC abstract.
Here is a good place to have something abstract-like that we progressively update to more fully describe the work. We should keep it clear and accessible, providing only the relevant summary information and avoiding unnecessary erudition or technicality.

# Datasets

The FEC dataset consists of around 500k face image triplets (derived from 156k images; see Table 1) along with annotation of which pair within the triplet are most visually similar in terms of facial expression at the exclusion of factors such as identity, gender, ethnicity, pose age, etc. (as assigned by multiple human annotators - six for the majority of the dataset). Triplets are also annotated according to how far different they are expected to be based on semantic expression labels (obtained by the authors separately and not distributed). These labels include Amusement, Anger, Awe, Boredom, Concentration, Confusion, Contemplation, Contempt, Contentment, Desire, Disappointment, Disgust, Distress, Doubt, Ecstasy, Elation, Embarrassment, Fear, Interest, Love, Neutral, Pain, Pride, Realization, Relief, Sadness, Shame, Surprise, Sympathy, Triumph. A one-class triplet is one where all three share a label in common; a two-class triplet is one where at most two share a label in common; and a three class triplet is one where none share a label. The dataset consists of such annotations, the URL of the publicly-available source image (primarily Flickr), and bounding boxes for the region within the source image where the annotated face image can be found.


<table style="width:100%">
  <tr>
    <th>Partition</th>
    <th>One-class</th>
    <th>Two-class</th>
    <th>Three-class</th>
    <th>All</th>
    <th>Faces</th>
  </tr>
  <tr>
    <td>Training</td>
    <td>152,674</td>
    <td>150,234</td>
    <td>146,235</td>
    <td>449,142</td>
    <td>130,516</td>
  </tr>
  <tr>
    <td>Test</td>
    <td>17,059</td>
    <td>17,107</td>
    <td>16,894</td>
    <td>51,060</td>
    <td>25,427</td>
  </tr>
  <tr>
    <td>Full</td>
    <td>169,733</td>
    <td>167,341</td>
    <td>163,129</td>
    <td>500,203</td>
    <td>155,943</td>
  </tr>
</table>

Table 1. Number of examples in FEC dataset by partition and triplet type.


For each triplet in the dataset, the download of each image was attempted and if successful filtered according to whether a face was present at the expected bounding box coordinates. Face detection was performed using Faced (https://github.com/iitzco/faced) which employs deep CNN’s, one stage of which includes a network based on YOLOv3 (Redmon and Farhadi, 2018), for fast face detection on CPUs. Images are first cropped to bounds provided by FEC (expanded by 15% along each axis) the result of which was passed to Faced for face detection and subsequent filtering on that basis. All images were then standardized to a size of 128px preserving aspect ratios and padding with gaussian noise.

The VGGFace2 dataset (Cao, et al., 2018) consists of 3.31M images from 9,131 individuals obtained via Google Image Search. These images span a wide variation in pose, age, illumination, and vocation. Naturally, the dataset is biased in abundance towards existing bias in the distribution of celebrities and public figures such as in terms of ethnicity and vocation. The gender balance of the dataset is 59.3% / 40.7% male/female. The dataset was designed to include a large number of images per identity with the min, mean, and max of these being 80, 362.6, and 843, respectively. Automated annotations include facial bounding boxes, facial keypoints, and age and pose predictions. Identity annotations were a mixture of initial search according to KnowledgeGraph name queries followed by filtering performed by human labelers on the basis of whether at least 90% of the images in an identity class were from the same individual.

# Methods

### Learning methods

Optimization was performed using the Adafactor optimizer (Shazeer and Stern, 2018) and networks were built with Trax (TODO: citation?) and where relevant were trained on Google Cloud TPUs.

### Vision networks

Networks for processing images were in all cases based passing the output of ResNet-50 into a series of fully connected layers (interspersed with ReLU activations) and ultimately L2 normalization to produce the output embedding. (TODO: Currently don’t reduce output of ResNet to a compact embedding as in Vemulapalli and Agarwala, 2019. Such reduction in dimensionality may be beneficial for query performance.)

In [0]:
# TODO: Import vision networks in the vision networks section or keep things more consolidated?

### Augmentation

Across all experiments, standard image augmentation techniques were applied including random cropping (randomly ~15% by area), horizontal flipping (p=0.5), and jittering of brightness and contrast (+/-10%). Audio was augmented by randomly scaling of volume by 10% in addition to sampling in random shifts as discussed below (the audio-domain analog of random cropping).

###### Demonstration

In [0]:
# TODO: Provide an illustration of means of performing and the result of these
# augmentation procedures.

### Triplet distance loss

For training on both the FEC expression similarity problem and pre-training on the VGGFace2 identity problem we use the triplet loss (1) advocated by Schroff, Kalenichenko, and Philbin (2015), reformulated by Agarwala and Vemulapalli (2019) to consider either I_1 or I_2 as anchor points (2), both encouraging the distance between positive pairs to be less than negative ones. The training time loss for SGD is computed in the standard way as the sum of the element-wise losses over the batch normalized by the batch size.

TODO: Display of image fec loss formulae in markdown.

l(I1,I2,I3) = d(eI1,eI2) - d(eI1,eI3)  + 𝛿      (1)

tpl(B)=1|B|tB max(0, d(eI1,eI2) - d(eI1,eI3) + 𝛿) + max(0, d(eI1,eI2) - d(eI2,eI3) + 𝛿 
for triplet t = (I1,I2,I3)       (2)



Here d(ei,ej) is the l2distance and 𝛿 is a margin of 0.1. Geometrically, for a single triplet, the latter can be imagined (Figure 1) as the maximum intersection two spherical functions (taking larger values at their center) of radius d(eI1,eI2) around eI1 and eI2plus a 𝛿-width envelope. The loss is zero anywhere outside of this domain.

### Embedding consistency learning

Some experiments were supplemented with additional unimodal self-supervision in the form introduced by Qizhe, et al. (2019; “unsupervised data augmentation” or, here, UDA) where an image I is differentially (randomly) augmented in two ways, giving I'and I'', the distance between the embeddings of these is sought to be minimized over batch B, uda(B)=1|B|IBd(eI',eI'').
TODO: Embedding consistency formula display in markdown.

### Pre-training on VGGFace2

Prior to training on the problem of facial expression correspondence, for which a comparatively much smaller dataset is available, networks were pre-trained in the style of FaceNet (and related) on the problem of identity understanding using the triplet embedding loss discussed above. Here l(I1,I2,I3) refers to I1 and I2 being images from the same person and I3 being an image from a person of a different identity. Here, intuitively, the loss encourages the distance between images of the same person to be smaller than that between those from different people. Training on identity recognition was terminated once evaluation accuracy reached 80% (i.e. training was not performed up to the level of elsewhere-reported accuracies 99%+).

# Experiments

### Learning facial expression correspondence

Training on facial expression correspondence was performed using examples from the FEC dataset based on pre-training on the aforementioned identity learning problem using the VGGFace2 dataset. In this case, the same triplet distance loss was used but here l(I1,I2,I3) refers to I1 and I2 being images human raters considered to be more similar in expression than either of those compared to I3. In contrast to the approach described in Vemulapalli and Agarwala (2019) we used the same 𝛿 margin of 0.1 for each of the triplet types and do not balance triplet types within batches. We further sought to understand the usefulness of UDA (Qizhe, et al., 2019) as an additional supervisory signal.

Triplet embedding accuracy (4) was computed in the way described in Vemulapalli and Agarwala (2019),

tpa(B)=1|B|tB( d(eI1,eI2)  d(eI1,eI3) ) & ( d(eI1,eI2)  d(eI2,eI3) ) for triplet t = (I1,I2,I3)    (4)
TODO: Markdown display of image FEC tpa metric.

where T is a collection of triplets, |T| is the size of that collection, and where d(ei,ej) is the l2distance used in training. Results for pre-training conditions are summarized in Table N and Figure N and for FEC transfer conditions summarized in Table N. and Figure N.

##### Table N. Triplet prediction accuracy by inclusion of UDA.
Triplet prediction accuracies are shown for a series of training iterations both with and without the inclusion of UDA self-supervision.

In [0]:
# TODO: A table showing the experimental results

##### Figure N. Triplet prediction accuracy and training loss over time.
Training loss (left) and triplet prediction accuracy (right) are shown over the course of training for both the UDA and non-UDA conditions (TODO: for now this is only the non-UDA condition). Notably, TPA grows above 0.64 then converges to approximately 0.61 suggesting improvements can be gained from modifications to training methods.

In [0]:
# TODO: Compute figures from CSV of training accuracy and loss data.

### Qualitative analysis

A sample of 2,000 training set images were embedded by the (TODO: best performing, still undetermined) model and processed for k-nearest-neighbors lookup using `sklearn.neighbors.KDTree` using a leaf size of 4 and the Euclidean metric. Search was performed with a k of 4 using an image from the evaluation set (not included in training). Query/result sets were filtered according to those where the top-ranked result had a distance of at most 1.77 as well as to omit images for the protection of the pictured (on account of apparent age or emotion expressed, e.g. malice). These results are shown in Figures N and N below.

Naturally populating the search space with only 2k images will tend to yield results of lower quality than one with 10 or 100 times that which is a near next step. Nevertheless for this configuration and despite triplet prediction accuracy reaching only about 0.61 the similarity in emotion state between query and low-distance results seems high.

In some cases, several copies of the same image will be returned as results for the same query. This is due to embeddings for these being computed on the augmented forms of the images. Relatedly, in some cases a query image will return an identical image as a top-ranked result but with non-zero distance. One point is that we would prefer embeddings to be invariant to augmentation so that when models are used in production their representations are invariant to dynamical changes in lighting, for example. This is also relevant for specifying (and cueing for) goals that are similarly independent of various dimensions of augmentation. Such invariance is likely obtainable by including the UDA loss component discussed above Another key issue raised by these observations is that while the triplets present in the training and eval sets may be disjoint, it appears the primary images are not. Thus an important next step is to construct a query set by cross-referencing the source image URLs of potential queries and those for images used in training.


##### Figure N. Qualitative analysis of FEC-trained model (part 1). 
Queries (left columns) and results (right columns) are shown for a variety of queries qualitatively illustrating model performance across a range of inputs.

In [0]:
# TODO: Compute and display QA results

# Future directions

TODO: Image FEC future directions. It will be nice to describe what some simple next directions are for this work so that those who want to build upon it have some guidance.

# References

# License

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License")
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

 https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.